# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [114]:
#Libraries
import pandas as pd
import numpy as np
import scipy.stats as st

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score

import optuna
import optuna.visualization as vis
import time




In [115]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [116]:
spaceship.dropna(how="any", inplace=True)

spaceship["Cabin"] = spaceship["Cabin"].apply(lambda x: x[0])

spaceship = spaceship.drop(columns = ["PassengerId", "Name"], axis=1)

target = spaceship["Transported"]

numerical = spaceship.select_dtypes(include="number")

scaler = StandardScaler()
scaler.fit(numerical)

standarized_numerical_np = scaler.transform(numerical)
standarized_numerical_df = pd.DataFrame(standarized_numerical_np, columns = numerical.columns, index=numerical.index)

non_numerical = spaceship.select_dtypes(exclude="number")
non_numerical = non_numerical.drop(columns=["Transported"], axis=1)
ohe = OneHotEncoder(sparse_output=False)

ohe.fit(non_numerical)

non_numerical_np = ohe.transform(non_numerical)

non_numerical_df = pd.DataFrame(non_numerical_np, columns=ohe.get_feature_names_out(), index=non_numerical.index)
features = pd.concat([standarized_numerical_df, non_numerical_df], axis=1)


In [117]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [118]:
gb_reg = GradientBoostingRegressor(max_depth=5, n_estimators=50)
gb_reg.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=5, n_estimators=50)

- Evaluate your model

In [119]:
y_pred_test_gb = gb_reg.predict(X_test)

print(f"MAE, {mean_absolute_error(y_pred_test_gb, y_test): .2f}")
print(f"MSE, {mean_squared_error(y_pred_test_gb, y_test): .2f}")
print(f"RMSE, {root_mean_squared_error(y_pred_test_gb, y_test): .2f}")
print(f"R2 score, {gb_reg.score(X_test, y_test): .2f}")

MAE,  0.28
MSE,  0.14
RMSE,  0.38
R2 score,  0.43


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [120]:
# First we need to setup a dictionary with all the values that we want to try for each hyprerparameter

parameter_grid = {"max_depth": [10, 50],
                  "min_samples_split": [4, 16],
                  "max_leaf_nodes": [250, 100],
                  "max_features": ["sqrt", "log2"]} # In example we're going to test 2 * 2 * 2 * 2 = 16 combinations of hyperparameters

# We create an instance or our machine learning model
dt = GradientBoostingRegressor()

# We need to set this two variables to be able to compute a confidence interval
confidence_level = 0.95
folds = 10




- Run Grid Search

In [122]:
# Now we need to create an intance of the GridSearchCV class
gs = GridSearchCV(dt, param_grid=parameter_grid, cv=folds, verbose=10) # Here the "cv" allows you to define the number of folds to use.

start_time = time.time()
gs.fit(X_train, y_train)
end_time = time.time()

print("\n")
print(f"Time taken to find the best combination of hyperparameters among the given ones: {end_time - start_time: .4f} seconds")
print("\n")


print(f"The best combination of hyperparameters has been: {gs.best_params_}")
print(f"The R2 is: {gs.best_score_: .4f}")

results_gs_df = pd.DataFrame(gs.cv_results_).sort_values(by="mean_test_score", ascending=False)

display(results_gs_df.head())

#print(results_df.head())
gs_mean_score = results_gs_df.iloc[0,-3]
gs_sem = results_gs_df.iloc[0,-2] / np.sqrt(10) # (std / sqrt(folds))

# Getting the critical value
gs_tc = st.t.ppf(1-((1-confidence_level)/2), df=folds-1)
gs_lower_bound = gs_mean_score - ( gs_tc * gs_sem )
gs_upper_bound = gs_mean_score + ( gs_tc * gs_sem )

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV 1/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 1/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.386 total time=   0.2s
[CV 2/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 2/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.464 total time=   0.2s
[CV 3/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 3/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.458 total time=   0.2s
[CV 4/10; 1/16] START max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4
[CV 4/10; 1/16] END max_depth=10, max_features=sqrt, max_leaf_nodes=250, min_samples_split=4;, score=0.427 total time=   0.2s
[CV 5/10; 1/16] START max_depth=10, max_features=sqrt

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_max_leaf_nodes,param_min_samples_split,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
2,0.141339,0.003657,0.001718,0.000047,10,sqrt,100,4,"{'max_depth': 10, 'max_features': 'sqrt', 'max...",0.405141,...,0.445761,0.429887,0.462646,0.431018,0.388502,0.473099,0.383767,0.439830,0.037018,1
14,0.135601,0.004271,0.001611,0.000078,50,log2,100,4,"{'max_depth': 50, 'max_features': 'log2', 'max...",0.414060,...,0.457030,0.412070,0.447402,0.430813,0.389863,0.477897,0.380864,0.439235,0.038163,2
10,0.134632,0.004417,0.001597,0.000073,50,sqrt,100,4,"{'max_depth': 50, 'max_features': 'sqrt', 'max...",0.399849,...,0.450281,0.401998,0.462626,0.430962,0.400047,0.469873,0.398186,0.438138,0.034342,3
3,0.161593,0.007863,0.002064,0.000206,10,sqrt,100,16,"{'max_depth': 10, 'max_features': 'sqrt', 'max...",0.401966,...,0.452309,0.400139,0.451938,0.438708,0.383134,0.469068,0.393672,0.438056,0.039541,4
7,0.158102,0.006956,0.001945,0.000091,10,log2,100,16,"{'max_depth': 10, 'max_features': 'log2', 'max...",0.387494,...,0.459902,0.418588,0.440946,0.433339,0.403952,0.464337,0.380154,0.436364,0.036857,5


- Evaluate your model

In [123]:


print(f"The R2 confidence interval for the best combination of hyperparameters is: \
    ({gs_lower_bound: .4f}, {gs_mean_score: .4f}, {gs_upper_bound: .4f}) ")

#display(results_df)

# Let's store the best model
best_model = gs.best_estimator_

# Now is time evaluate the model in the test set
y_pred_test_df = best_model.predict(X_test)
y_pred_test_df = best_model.predict(X_test)

y_pred_test_df = best_model.predict(X_test)

print("\n")
print(f"Test MAE: {mean_absolute_error(y_pred_test_df, y_test): .4f}")
print(f"Test MSE: {mean_squared_error(y_pred_test_df, y_test): .4f}")
print(f"Test RMSE: {root_mean_squared_error(y_pred_test_df, y_test): .4f}")
print(f"Test R2 score:  {best_model.score(X_test, y_test): .4f}")
print("\n")


The R2 confidence interval for the best combination of hyperparameters is:     ( 0.4133,  0.4398,  0.4663) 


Test MAE:  0.2731
Test MSE:  0.1433
Test RMSE:  0.3785
Test R2 score:   0.4268




In [121]:
# The best combination of hyperparameters has been: {'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': 100, 'min_samples_split': 4}
# The R2 is:  0.4402

# The R2 confidence interval for the best combination of hyperparameters is:     ( 0.4076,  0.4402,  0.4727) 

# Test MAE:  0.2771
# Test MSE:  0.1456
# Test RMSE:  0.3815
# Test R2 score:   0.4178